In [5]:
# https://www.kaggle.com/code/sarmat/lgbm-stacking-example/notebook

In [87]:
import numpy as np # linear algebra
import pandas as pd
import random
from scipy.sparse.linalg import svds

from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split

import torch
from tqdm import tqdm
from dataset import custom_train_test_split, make_dataset

from sklearn.metrics import RocCurveDisplay, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc


def get_metric(targets, preds):
    auc = roc_auc_score(targets, preds)
    acc = accuracy_score(targets, np.where(preds >= 0.5, 1, 0))
    precsion = precision_score(targets, np.where(preds >= 0.5, 1, 0))
    recall = recall_score(targets, np.where(preds >= 0.5, 1, 0))
    F1_score = f1_score(targets, np.where(preds >= 0.5, 1, 0))

    print('auc :',auc)
    print('acc :',acc)
    print('precision :',precsion)
    print('recall :',recall)

def test_to_csv(preds, name:str):
    
    result = []
    for n,i in enumerate(preds):
        row = {}    
        row['id'] = n
        row['prediction'] = i
        result.append(row)
    pd.DataFrame(result).to_csv(f'output/{name}.csv', index=None)
    


In [7]:
train_data = pd.read_csv('/opt/ml/input/data/train_data.csv')
test_data  = pd.read_csv('/opt/ml/input/data/test_data.csv')

In [8]:
train_data.drop_duplicates(subset = ["userID", "assessmentItemID"], keep = "last", inplace = True)
train_data.drop(['Timestamp','testId','KnowledgeTag'], axis=1, inplace=True, errors='ignore')

In [9]:
matrix_train = train_data.pivot_table('answerCode', index='userID', columns='assessmentItemID')
matrix_train.fillna(0.5, inplace=True)

In [10]:
user_id2idx = {v:i for i,v in enumerate(matrix_train.index)}
user_idx2id = {i:v for i,v in enumerate(matrix_train.index)}

item_id2idx = {v:i for i,v in enumerate(matrix_train.columns)}
item_idx2id = {i:v for i,v in enumerate(matrix_train.columns)}

In [16]:
def predict(matrix, userid, itemid, user_id2idx, item_id2idx, k):
    A = matrix_train.values
    a_mean = np.mean(A, axis=1)
    Am = A - a_mean.reshape(-1,1)

    U, sigma, V = svds(Am, k=k)
    
    Sigma = np.diag(sigma)
    Sigma_i = np.diag(1/sigma)
    pred_matrix = V.T @ Sigma_i @ Sigma @ V
    
    B = matrix
    B_mean = np.mean(B, axis=1)
    Bm = B - B_mean.reshape(-1,1)

    B_pred =  B @ pred_matrix + B_mean.reshape(-1,1)

    ret = [B_pred[user_id2idx[u], item_id2idx[i]] for u,i in zip(userid, itemid)]
    return ret

In [68]:
valid_user  = pd.read_csv('/opt/ml/input/data/cv_valid_data.csv').userID.unique()
all_train = pd.read_csv('/opt/ml/input/data/all.csv')
valid_data = all_train[all_train.userID.isin(valid_user)]
userid = sorted(list(set([u for u in valid_data.userID])))
user_id2idx_valid = {v:i for i,v in enumerate(userid)}

matrix_valid = 0.5*np.ones((len(userid), len(item_id2idx)))
for user,item,a in zip(valid_data.userID, valid_data.assessmentItemID, valid_data.answerCode):
    user,item = user_id2idx_valid[user],item_id2idx[item]
    matrix_valid[user,item] = a

valid_predict1 = predict(matrix_valid, valid_data.userID, valid_data.assessmentItemID, user_id2idx_valid, item_id2idx, 6)
valid_predict2 = predict(matrix_valid, valid_data.userID, valid_data.assessmentItemID, user_id2idx_valid, item_id2idx, 8)
valid_predict3 = predict(matrix_valid, valid_data.userID, valid_data.assessmentItemID, user_id2idx_valid, item_id2idx, 10)
valid_predict4 = predict(matrix_valid, valid_data.userID, valid_data.assessmentItemID, user_id2idx_valid, item_id2idx, 12)
valid_predict5 = predict(matrix_valid, valid_data.userID, valid_data.assessmentItemID, user_id2idx_valid, item_id2idx, 14)

# item_id2idx는 train에서 사용한 것을 다시 사용한다.
test_data  = pd.read_csv('/opt/ml/input/data/test_data.csv')

userid = sorted(list(set([u for u in test_data.userID])))
user_id2idx_test = {v:i for i,v in enumerate(userid)}

matrix_test = 0.5*np.ones((len(userid), len(item_id2idx)))
for user,item,a in zip(test_data.userID, test_data.assessmentItemID, test_data.answerCode):
    user,item = user_id2idx_test[user],item_id2idx[item]
    matrix_test[user,item] = a

test_data = test_data[test_data.answerCode==-1]

test_predict1 = predict(matrix_test, test_data.userID, test_data.assessmentItemID, user_id2idx_test, item_id2idx, 6)
test_predict2 = predict(matrix_test, test_data.userID, test_data.assessmentItemID, user_id2idx_test, item_id2idx, 8)
test_predict3 = predict(matrix_test, test_data.userID, test_data.assessmentItemID, user_id2idx_test, item_id2idx, 10)
test_predict4 = predict(matrix_test, test_data.userID, test_data.assessmentItemID, user_id2idx_test, item_id2idx, 12)
test_predict5 = predict(matrix_test, test_data.userID, test_data.assessmentItemID, user_id2idx_test, item_id2idx, 14)

# print('Fold no: {}'.format(fold_))
print("AUC SVD1:{} ".format(get_metric(valid_data.answerCode.to_numpy(), np.array(valid_predict1))))
print("AUC SVD2:{} ".format(get_metric(valid_data.answerCode.to_numpy(), np.array(valid_predict2))))
print("AUC SVD3:{} ".format(get_metric(valid_data.answerCode.to_numpy(), np.array(valid_predict3))))
print("AUC SVD4:{} ".format(get_metric(valid_data.answerCode.to_numpy(), np.array(valid_predict4))))
print("AUC SVD5:{} ".format(get_metric(valid_data.answerCode.to_numpy(), np.array(valid_predict5)))) 

auc : 0.7594001217435203
acc : 0.7453792462709189
precision : 0.7645841458024595
recall : 0.8834209498633182
AUC SVD1:None 
auc : 0.7668146174036005
acc : 0.7531621899321699
precision : 0.7681388664888448
recall : 0.8927753912703896
AUC SVD2:None 
auc : 0.7737004979789429
acc : 0.7570398831574264
precision : 0.773271196789496
recall : 0.8902340112349425
AUC SVD3:None 
auc : 0.7792546574725867
acc : 0.759894022840991
precision : 0.7752936876729494
recall : 0.892138544263871
AUC SVD4:None 
auc : 0.7830752291494566
acc : 0.7623741718073987
precision : 0.7773176903611686
recall : 0.8932440145393373
AUC SVD5:None 


In [90]:
new_valid = np.array([valid_predict1, valid_predict2, valid_predict3, valid_predict4, valid_predict5]).T

new_test = np.array([test_predict1, test_predict2, test_predict3, test_predict4, test_predict5]).T

In [83]:
val = pd.read_csv('/opt/ml/input/data/cv_valid_data.csv')
tail_idx = val.index[val.answerCode==-1].to_numpy()

In [91]:
y_valid = valid_data.answerCode.to_numpy()

valid_tail = [new_valid[i] for i in range(len(new_valid)) if i in tail_idx]
y_tail = [y_valid[i] for i in range(len(y_valid)) if i in tail_idx]

new_valid = [new_valid[i] for i in range(len(new_valid)) if not i in tail_idx]
y_new_valid = [y_valid[i] for i in range(len(y_valid)) if not i in tail_idx]

In [99]:
from catboost import CatBoostClassifier, Pool

train_pool = Pool(new_valid, y_new_valid)
# eval_pool = Pool(valid_tail , y_tail)
eval_pool = Pool(valid_tail, y_tail)

Final_cat = CatBoostClassifier(
            iterations = 3000,
            random_seed = 42,
            learning_rate = 0.0005,
            loss_function = 'Logloss', 
            custom_metric = ['Logloss','AUC'],
            early_stopping_rounds = 30,
            use_best_model =  True,
            task_type = "GPU",
            bagging_temperature = 1,
            verbose = False)

Final_cat.fit(train_pool, eval_set=eval_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [102]:
preds = Final_cat.predict(new_test , prediction_type='Probability')[:,1]

from datetime import date, datetime, timezone, timedelta

KST = timezone(timedelta(hours=9))
time_record = datetime.now(KST)
_day = str(time_record)[:10]
_time = str(time_record.time())[:8]
now_time = _day+'_'+_time

test_to_csv(new_test.mean(axis=1),f'Stacking_SVD_{now_time}')

744